<a href="https://colab.research.google.com/github/DayDreamChaser/llm-todo/blob/main/LLM/Llama3_8B_instrcut_DemoTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import login
login()


In [22]:

import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
  "text-generation",
  model="meta-llama/Meta-Llama-3-8B-Instruct",
  model_kwargs={"torch_dtype": torch.bfloat16},
  device="cuda",
)

In [1]:
!pip install --upgrade accelerate
!pip install bitsandbytes transformers_stream_generator
# !pip install transformers
# !pip install torch
# !pip install accelerate

  Using cached accelerate-0.29.3-py3-none-any.whl (297 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 k

In [2]:
import accelerate
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer

# 待加载的预模型
model_path = "meta-llama/Meta-Llama-3-8B-Instruct"

# 分词器
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
        model_path,
        load_in_4bit=True,
        torch_dtype=torch.float16,
        device_map='auto'
    )
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [42]:
from sys import getsizeof as getsize

print(getsize(model))

48


In [41]:
type(model)

transformers.models.llama.modeling_llama.LlamaForCausalLM

In [47]:
def get_model_size(model):
	para_num = sum([p.numel() for p in model.parameters()])
	# para_size: 参数个数 / 1024 / 1024，单位为 MB
	para_size = para_num / 1024 / 1024
	return para_size

get_model_size(model) / 1024

4.228763580322266

In [44]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

In [16]:
instruction = """<|start_header_id|>system\n{system}<|end_header_id|>
<|begin_of_text|>user\n{input}<|end_header_id|>\n<|begin_of_text|>assistant"""  # \n<|end_header_id|>
print(instruction)

<|start_header_id|>system
{system}<|end_header_id|>
<|begin_of_text|>user
{input}<|end_header_id|>
<|begin_of_text|>assistant


In [17]:
system_words = "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."

prompt = instruction.format(system=system_words, input="What is the meaning of life")
print(prompt)

<|start_header_id|>system
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<|end_header_id|>
<|begin_of_text|>user
What is the meaning of life<|end_header_id|>
<|begin_of_text|>assistant


In [18]:
generate_ids = model.generate(tokenizer(prompt, return_tensors='pt').input_ids.cuda(), max_new_tokens=4096, streamer=streamer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



What a profound and timeless question!

The meaning of life is a topic that has been debated and explored by philosophers, scientists, spiritual leaders, and individuals from all walks of life for centuries. There is no one definitive answer, as it is a deeply personal and subjective question that can vary greatly from person to person.

For some, the meaning of life may be found in their relationships with others, whether romantic, familial, or platonic. For others, it may be discovered through their work, passions, or contributions to society. Some may find meaning in their spiritual beliefs or practices, while others may derive it from their experiences, achievements, or personal growth.

In many cultures and traditions, the meaning of life is often tied to a sense of purpose, fulfillment, or happiness. It may be seen as a journey of self-discovery, where individuals explore their values, goals, and aspirations to find direction and significance.

Ultimately, the meaning of life is

In [25]:
system_words = "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."

def chat(user_input):
  prompt = instruction.format(system=system_words, input=user_input)
  generate_ids = model.generate(tokenizer(prompt, return_tensors='pt').input_ids.cuda(), max_new_tokens=4096, streamer=streamer)

model.config.pad_token_id = model.config.eos_token_id

In [39]:
chat("你好，周末2天怎么安排，学习大模型和计算机")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



😊

Hello! It's great that you're planning your weekend to focus on learning and improving your skills in machine learning and computer science. Here's a suggested plan for you:

**Day 1:**

1. **Morning**: Start with a review of the basics of machine learning and computer science. You can refresh your memory by going through online resources like Coursera, edX, or Khan Academy.
2. **Afternoon**: Choose a specific topic you want to explore, such as deep learning, natural language processing, or computer vision. Read articles, research papers, or blog posts on that topic to gain a deeper understanding.
3. **Evening**: Practice what you've learned by working on a project or contributing to an open-source project related to machine learning or computer science.

**Day 2:**

1. **Morning**: Focus on a specific skill or tool you want to master, such as Python programming, TensorFlow, or Keras. Practice coding exercises or work on a project that utilizes that skill.
2. **Afternoon**: Watch o

In [27]:
chat("写一首关于大雨和落日的古诗")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Here's a traditional Chinese poem about a big rain and sunset:

大雨如注，落日如焰，
天地同沸，人间同欢。
雨过天晴，日落山深，
风雨同归，人心同宁。

Translation:

The big rain pours down like a torrent, the sunset blazes like a flame,
Heaven and earth are in turmoil, and the world is in jubilation.
After the rain clears, the sky is bright, and the sun dips into the mountains,
The wind and rain return to their source, and the human heart finds peace.

This poem is inspired by the traditional Chinese poetry style, with a focus on the natural world and the emotions it evokes. The poem describes the intense energy of a big rain and the vibrant colors of a sunset, and how they bring people together and bring peace to the human heart.


In [31]:
text = """### Instruction ###
给定如下马列主义原著，请规划一个合理循序渐进的学习路线，必须考虑每一本书，并给出理由(必须用中文回答)

### Input ###
一、马恩原著
哲学
1《路德维希费尔巴哈和德国古典哲学的终结》
2《反杜林论》

政治经济学
3《1844年经济学哲学手稿》
4《政治经济学批判导言序言》
5《雇佣劳动与资本》
6《工资价格与利润》

社科类
7《共产党宣言》
8《论住宅问题》
9《哥达纲领批判》
10《1848-1850年法兰西的阶级斗争》
11《法兰西内战》
12《路易波拿巴的雾月十八日》
13《家庭私有制和国家的起源》

二、列宁著作
1《国家与革命》
2《帝国主义是资本主义的最高阶段》
3《唯物主义与经验批判主义》
4《社会民主党在民主革命中的两种策略》
5《共产主义运动中的“左”派幼稚病》

### Output ###
"""

In [29]:
chat(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



I'd be happy to help you plan a logical and progressive learning path through the Marxist classics. Here's a suggested reading order, considering each book and providing reasons:

**Phase 1: Understanding the Foundations**

1. 《路德维希费尔巴哈和德国古典哲学的终结》 (The Holy Family) - This book provides a critique of German classical philosophy, which is essential for understanding Marx's philosophical foundations.
2. 《反杜林论》 (The German Ideology) - This book is a continuation of the previous one, further developing Marx's philosophical ideas.

**Phase 2: Economic and Political Analysis**

1. 《1844年经济学哲学手稿》 (Economic and Philosophic Manuscripts of 1844) - This book introduces key concepts like alienation, commodity fetishism, and the labor theory of value.
2. 《政治经济学批判导言序言》 (Critique of Political Economy) - This book provides a comprehensive analysis of political economy, laying the groundwork for Marx's later works.
3. 《雇佤劳动与资本》 (Wage Labor and Capital) - This book explores the relationship between wage

In [32]:
chat(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


system
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
user
### Instruction ###
给定如下马列主义原著，请规划一个合理循序渐进的学习路线，必须考虑每一本书，并给出理由(必须用中文回答)

### Input ###
一、马恩原著
哲学
1《路德维希费尔巴哈和德国古典哲学的终结》
2《反杜林论》

政治经济学
3《1844年经济学哲学手稿》
4《政治经济学批判导言序言》
5《雇佣劳动与资本》
6《工资价格与利润》

社科类
7《共产党宣言》
8《论住宅问题》
9《哥达纲领批判》
10《1848-1850年法兰西的阶级斗争》
11《法兰西内战》
12《路易波拿巴的雾月十八日》
13《家庭私有制和国家的起源》

二、列宁著作
1《国家与革命》
2《帝国主义是资本主义的最高阶段》
3《唯物主义与经验批判主义》
4《社会民主党在民主革命中的两种策略》
5《共产主义运动中的“左”派幼稚病》

### Output ###

assistant
Here is a suggested learning path for the Marxist classics, considering each boo

In [ ]:
system_words = "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."

def chat(user_input):
  prompt = instruction.format(system=system_words, input=user_input)
  generate_ids = model.generate(tokenizer(prompt, return_tensors='pt').input_ids.cuda(), max_new_tokens=4096, streamer=streamer)


In [66]:
trans_text = "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."


translate_prompt_ch = """### Instruction ###
请将如下文<>括号内的英文翻译为中文，必须保证准确、流程、精简(必须只输出翻译结果)

### Input ###
英文: <{trans_text}>

### Output ###
翻译结果：
"""

In [68]:
chat(translate_prompt_ch.format(trans_text=trans_text))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|start_header_id|>system
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<|end_header_id|>
<|begin_of_text|>user
### Instruction ###
请将如下文<>括号内的英文翻译为中文，必须保证准确、流程、精简(必须只输出翻译结果)

### Input ###
英文: <You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is 

In [63]:
translate_prompt = """### Instruction ###
请将如下文<>括号内的英文翻译为中文，必须保证准确、流程、精简

### Input ###
英文: <{trans_text}>

### Output ###
翻译结果：
"""

In [64]:
chat(translate_prompt.format(trans_text=text))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|start_header_id|>system
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<|end_header_id|>
<|begin_of_text|>user
### Instruction ###
请将如下文<>括号内的英文翻译为中文，必须保证准确、流程、精简

### Input ###
英文: <### Instruction ###
请简洁精炼地总结如下文通信商客服与客户对话的摘要
要求：1、必须保证准确、避免篡改和追加对话中没有地信息，必须将字数限制在150字以内
   2、摘要输出必须是中文

### Input ###
对话: "{dialogs}"

### Output ###
摘要：
>

### Output ###
翻译结果：
<|end_header_id|>
<|begin_of_text|>assistant

I'd be happy to help! However, I noticed that there is no text within the <> brackets. Could you please provide the English text you

In [48]:
x = """【坐席】您好，实习话务员为您，【客户】服务，喂，你好，是这样的，我那个我的电话因为上次突然把给我停机了，我不知道啥意思，然后我就好长时间没有使用，我这次拿着10，怎么我欠费160多了？噢，我想问一下，我这机器现在是在停机状态吧，【坐席】嗯，对是的？【客户】对停机状态，你们怎么还收我费用啊，【坐席】嗯，您稍等这边看的话，先生，咱们这12月份的话是没有收您的费用的，这边是这个产生的费用是您11月份的女士先生，【客户】那我11月份我现在-160，那我想问一下，那我这个怎么能-160？我始终没超一个星期，他就你们就给我停机了，因为那我忙着在外地出差，我就没有，我就没有那个办这个找你们，【坐席】嗯，先生这边看的话，先生，您是一个就是11月份的话，有一个是，嗯，基本套餐费是88块钱，还有您上个月未结清的话费是76块钱，【客户】但是我刚才你们给我反馈那个是我欠160多，【坐席】嗯，对是的先生，就是您这160多，嗯，160多是产生的，是11月份的结转11月份的，总消费是88，还有您10月还有一个是10月份未结清的话，是76块钱？【客户】我没有结清的，我就想问你，你帮我查一下，我打几天电话，我哪来的那个？还有那么多没结清呢，我就不明白，我一个包月电话我都没有打出去多少电话，我这流我这是4000分钟，是两三千分钟啊，那那么多话费我根本都没有打几个，他怎么就我又欠那么多钱呢【坐席】嗯，先生，您这个是阿里大包，嗯，就是三角大大宝卡吗？先生每个月基本月租费是88块钱的，里面是给您包含500分钟通话工作，工作员是二百两千五百分钟，相当于3000分钟吗？先生，还有两个G的流量，【客户】对呀，【坐席】嗯，它这个是如果您，嗯没有超出的话，每个月是收您88块钱的先生，【客户】对呀，那我也没有超出啊，你怎么还有我查，我这还欠那么多费呢【坐席】这嗯，就每个月正常扣您的套餐费用，先生88块钱套餐费用，先生这个是，【客户】对，但是我这个你看一下，你看看你从我开始，我注册这号到停机，您看看我使这几天你你对你们，你查一下看我使了几天，我怎么的就是有这个还有欠费呢，我这还没，我还说实在我找你们，我这使那几天，我那88块钱你们都给我退我，【坐席】嗯这边看的话是在9月份给您给您停的机先生，【客户】对啊，9月份我是9月几号开通的这个卡呀【坐席】嗯，您这是9月1号给您开通的卡先生，【客户】对9月份9月1号开通卡，应该没超过，9月10号，你就把我这个停了应该，【坐席】嗯是的先生，【客户】对呀，那你说那那您您觉得我这个怎么弄，你，你说我该怎么弄？您觉得你们问问你们公司领导，我这个怎么能查算合理呀【坐席】嗯，您看这样好吧先生，就是您这个问题的话，咱们这边给您记录反馈一下，稍后有咱们的专员给您回电，然后到底是这个问题如何给您解决？您看这样吧，先生【客户】行你，【坐席】就联系，嗯好联系电话联系您，哪个手机号码呢先生，【客户】你就联系这个就行，【坐席】嗯，这边的话不是已经是给您暂停服务了，先生这边就怕咱们联系您的话，也是无法无法接听电话的，先生无法打通的，【客户】他给我打那个185，185，1951【坐席】********，嗯，就联系您这个号码了，先生，【客户】4089对【坐席】嗯，您看还有什么问题可以帮您的吗？先生，【客户】没有了，你再给我把这给我解决一下吧，好吧，【坐席】嗯，好，嗯，好的先生，【客户】嗯【坐席】感谢您的来电，请您稍后对服务评价中帮我按个十分满意，谢谢|用户来电反映手机号卡收费问题，用户称现在怎么欠费167.65，我方经查询用户有10月欠费76和11月欠费88，用户称********此号卡是在9月1号开通的，没用几天就别关停服务了(局方停机），我都没有怎么使用，怎么还扣取我的费用，我某某人常解释，用户对超出的费用不认可，请核实处理，谢谢"""
x

'【坐席】您好，实习话务员为您，【客户】服务，喂，你好，是这样的，我那个我的电话因为上次突然把给我停机了，我不知道啥意思，然后我就好长时间没有使用，我这次拿着10，怎么我欠费160多了？噢，我想问一下，我这机器现在是在停机状态吧，【坐席】嗯，对是的？【客户】对停机状态，你们怎么还收我费用啊，【坐席】嗯，您稍等这边看的话，先生，咱们这12月份的话是没有收您的费用的，这边是这个产生的费用是您11月份的女士先生，【客户】那我11月份我现在-160，那我想问一下，那我这个怎么能-160？我始终没超一个星期，他就你们就给我停机了，因为那我忙着在外地出差，我就没有，我就没有那个办这个找你们，【坐席】嗯，先生这边看的话，先生，您是一个就是11月份的话，有一个是，嗯，基本套餐费是88块钱，还有您上个月未结清的话费是76块钱，【客户】但是我刚才你们给我反馈那个是我欠160多，【坐席】嗯，对是的先生，就是您这160多，嗯，160多是产生的，是11月份的结转11月份的，总消费是88，还有您10月还有一个是10月份未结清的话，是76块钱？【客户】我没有结清的，我就想问你，你帮我查一下，我打几天电话，我哪来的那个？还有那么多没结清呢，我就不明白，我一个包月电话我都没有打出去多少电话，我这流我这是4000分钟，是两三千分钟啊，那那么多话费我根本都没有打几个，他怎么就我又欠那么多钱呢【坐席】嗯，先生，您这个是阿里大包，嗯，就是三角大大宝卡吗？先生每个月基本月租费是88块钱的，里面是给您包含500分钟通话工作，工作员是二百两千五百分钟，相当于3000分钟吗？先生，还有两个G的流量，【客户】对呀，【坐席】嗯，它这个是如果您，嗯没有超出的话，每个月是收您88块钱的先生，【客户】对呀，那我也没有超出啊，你怎么还有我查，我这还欠那么多费呢【坐席】这嗯，就每个月正常扣您的套餐费用，先生88块钱套餐费用，先生这个是，【客户】对，但是我这个你看一下，你看看你从我开始，我注册这号到停机，您看看我使这几天你你对你们，你查一下看我使了几天，我怎么的就是有这个还有欠费呢，我这还没，我还说实在我找你们，我这使那几天，我那88块钱你们都给我退我，【坐席】嗯这边看的话是在9月份给您给您停的机先生，【客户】对啊，9月份我是9月几号开通的这个卡呀【坐席】嗯，您这是9月1号给您开通的卡先生，【客户】对9月份9月1号开通卡，应该没超

In [61]:
prompt1 = """### Instruction ###
请简洁精炼地总结如下文通信商客服与客户对话的摘要 （必须用中文回答结果)
要求：1、必须保证准确、避免篡改和追加对话中没有地信息，必须将字数限制在150字以内

### Input ###
对话: "{dialogs}"

### Output ###
摘要：
"""

In [56]:
llm_input = prompt1.format(dialogs=x)
print(llm_input)
chat(llm_input)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


### Instruction ###
请简洁精炼地总结如下文通信商客服与客户对话的摘要
要求：1、必须保证准确、避免篡改和追加对话中没有地信息，必须将字数限制在150字以内
   2、摘要输出必须是中文

### Input ###
对话: "【坐席】您好，实习话务员为您，【客户】服务，喂，你好，是这样的，我那个我的电话因为上次突然把给我停机了，我不知道啥意思，然后我就好长时间没有使用，我这次拿着10，怎么我欠费160多了？噢，我想问一下，我这机器现在是在停机状态吧，【坐席】嗯，对是的？【客户】对停机状态，你们怎么还收我费用啊，【坐席】嗯，您稍等这边看的话，先生，咱们这12月份的话是没有收您的费用的，这边是这个产生的费用是您11月份的女士先生，【客户】那我11月份我现在-160，那我想问一下，那我这个怎么能-160？我始终没超一个星期，他就你们就给我停机了，因为那我忙着在外地出差，我就没有，我就没有那个办这个找你们，【坐席】嗯，先生这边看的话，先生，您是一个就是11月份的话，有一个是，嗯，基本套餐费是88块钱，还有您上个月未结清的话费是76块钱，【客户】但是我刚才你们给我反馈那个是我欠160多，【坐席】嗯，对是的先生，就是您这160多，嗯，160多是产生的，是11月份的结转11月份的，总消费是88，还有您10月还有一个是10月份未结清的话，是76块钱？【客户】我没有结清的，我就想问你，你帮我查一下，我打几天电话，我哪来的那个？还有那么多没结清呢，我就不明白，我一个包月电话我都没有打出去多少电话，我这流我这是4000分钟，是两三千分钟啊，那那么多话费我根本都没有打几个，他怎么就我又欠那么多钱呢【坐席】嗯，先生，您这个是阿里大包，嗯，就是三角大大宝卡吗？先生每个月基本月租费是88块钱的，里面是给您包含500分钟通话工作，工作员是二百两千五百分钟，相当于3000分钟吗？先生，还有两个G的流量，【客户】对呀，【坐席】嗯，它这个是如果您，嗯没有超出的话，每个月是收您88块钱的先生，【客户】对呀，那我也没有超出啊，你怎么还有我查，我这还欠那么多费呢【坐席】这嗯，就每个月正常扣您的套餐费用，先生88块钱套餐费用，先生这个是，【客户】对，但是我这个你看一下，你看看你从我开始，我注册这号到停机，您看看我使这几天你你对你们，你查一下看我使了几天，我怎么的就是有这个还有欠费呢，我这还没

In [59]:
ch_system_words = "你是一个有用的、尊重的和诚实的助手。总是尽可能地帮助回答，同时保持安全。你的回答不应该包含任何有害、不道德、种族歧视、性别歧视、毒性、危险或非法内容。请确保你的回应是社会公正和积极的性质。如果问题不明确或不具事实依据，请解释为什么，而不是回答错误的内容。如果你不知道问题的答案，请不要分享假信息。（你需要用中文回答问题）"
def chat(user_input,system_words=system_words):
  prompt = instruction.format(system=system_words, input=user_input)
  print(prompt)
  generate_ids = model.generate(tokenizer(prompt, return_tensors='pt').input_ids.cuda(), max_new_tokens=4096, streamer=streamer)

In [62]:
llm_input = prompt1.format(dialogs=x)
print(llm_input)
chat(llm_input,system_words=ch_system_words)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


### Instruction ###
请简洁精炼地总结如下文通信商客服与客户对话的摘要 （必须用中文回答结果)
要求：1、必须保证准确、避免篡改和追加对话中没有地信息，必须将字数限制在150字以内
   
### Input ###
对话: "【坐席】您好，实习话务员为您，【客户】服务，喂，你好，是这样的，我那个我的电话因为上次突然把给我停机了，我不知道啥意思，然后我就好长时间没有使用，我这次拿着10，怎么我欠费160多了？噢，我想问一下，我这机器现在是在停机状态吧，【坐席】嗯，对是的？【客户】对停机状态，你们怎么还收我费用啊，【坐席】嗯，您稍等这边看的话，先生，咱们这12月份的话是没有收您的费用的，这边是这个产生的费用是您11月份的女士先生，【客户】那我11月份我现在-160，那我想问一下，那我这个怎么能-160？我始终没超一个星期，他就你们就给我停机了，因为那我忙着在外地出差，我就没有，我就没有那个办这个找你们，【坐席】嗯，先生这边看的话，先生，您是一个就是11月份的话，有一个是，嗯，基本套餐费是88块钱，还有您上个月未结清的话费是76块钱，【客户】但是我刚才你们给我反馈那个是我欠160多，【坐席】嗯，对是的先生，就是您这160多，嗯，160多是产生的，是11月份的结转11月份的，总消费是88，还有您10月还有一个是10月份未结清的话，是76块钱？【客户】我没有结清的，我就想问你，你帮我查一下，我打几天电话，我哪来的那个？还有那么多没结清呢，我就不明白，我一个包月电话我都没有打出去多少电话，我这流我这是4000分钟，是两三千分钟啊，那那么多话费我根本都没有打几个，他怎么就我又欠那么多钱呢【坐席】嗯，先生，您这个是阿里大包，嗯，就是三角大大宝卡吗？先生每个月基本月租费是88块钱的，里面是给您包含500分钟通话工作，工作员是二百两千五百分钟，相当于3000分钟吗？先生，还有两个G的流量，【客户】对呀，【坐席】嗯，它这个是如果您，嗯没有超出的话，每个月是收您88块钱的先生，【客户】对呀，那我也没有超出啊，你怎么还有我查，我这还欠那么多费呢【坐席】这嗯，就每个月正常扣您的套餐费用，先生88块钱套餐费用，先生这个是，【客户】对，但是我这个你看一下，你看看你从我开始，我注册这号到停机，您看看我使这几天你你对你们，你查一下看我使了几天，我怎么的就是有这个还有欠费呢，我这还没